# Exemple de notebook : Utilisation du package IBIS pour pré-filtrer, faire des jointures et aggréger avant de charger les données avec pandas

Avec la collecte de plusieurs années d’historique des données SISE, les volumes de données disponibles dans les tables edc_communes, edc_prelevements et edc_resultats vont significativement augmenter. Cette situationsituation pose des défis en termes de performances pour le chargement des données.

En particulier, une approche consistant à charger une table entière dans un DataFrame Pandas avec une commande comme :

```python
df = con.table("edc_resultats").df()
```

peut devenir lente et gourmande en mémoire.

Ce notebook a pour objectif de montrer :

- Comment utiliser [Ibis](https://ibis-project.org/tutorials/getting_started) pour interroger et filtrer efficacement les données avant de les charger dans Pandas.
- Les bonnes pratiques pour optimiser les performances en filtrant dès la source, sans importer inutilement des millions de lignes.
- Des exemples concrets adaptés à vos besoins d'analyse, comme le filtrage par dates, jointures entre tables ou la sélection de paramètres spécifiques.

In [39]:
import ibis
ibis_con = ibis.connect("./../../database/data.duckdb", read_only=True)

Pour commencer, listons les tables disponibles dans la base de données :

In [40]:
ibis_con.list_tables()

['edc_communes', 'edc_prelevements', 'edc_resultats']

L'inspection des colonnes d'une table peut se faire directement avec Ibis.

In [41]:
communes_table = ibis_con.table("edc_communes")
prelevements_table = ibis_con.table("edc_prelevements")
resultats_table = ibis_con.table("edc_resultats")

display(communes_table)
display(prelevements_table)
display(resultats_table)

DatabaseTable: edc_communes
  inseecommune string
  nomcommune   string
  quartier     string
  cdreseau     string
  nomreseau    string
  debutalim    date
  de_partition int32

DatabaseTable: edc_prelevements
  cddept                     string
  cdreseau                   string
  inseecommuneprinc          string
  nomcommuneprinc            string
  cdreseauamont              string
  nomreseauamont             string
  pourcentdebit              string
  referenceprel              string
  dateprel                   date
  heureprel                  string
  conclusionprel             string
  ugelib                     string
  distrlib                   string
  moalib                     string
  plvconformitebacterio      string
  plvconformitechimique      string
  plvconformitereferencebact string
  plvconformitereferencechim string
  de_partition               int32

DatabaseTable: edc_resultats
  cddept                   string
  referenceprel            string
  cdparametresiseeaux      string
  cdparametre              int64
  libmajparametre          string
  libminparametre          string
  libwebparametre          string
  qualitparam              string
  insituana                string
  rqana                    string
  cdunitereferencesiseeaux string
  cdunitereference         string
  limitequal               string
  refqual                  string
  valtraduite              float64
  casparam                 string
  referenceanl             string
  de_partition             int32

## Filtres

1. Filtrer sur les prélévements de 2024

In [51]:
prelevements_2024 = prelevements_table.filter(prelevements_table.dateprel >= "2024-01-01")
df_prelevements_2024 = prelevements_2024.limit(50).execute()
df_prelevements_2024

,cddept,cdreseau,inseecommuneprinc,nomcommuneprinc,cdreseauamont,nomreseauamont,pourcentdebit,referenceprel,dateprel,heureprel,conclusionprel,ugelib,distrlib,moalib,plvconformitebacterio,plvconformitechimique,plvconformitereferencebact,plvconformitereferencechim,de_partition
0,001,001000003,01007,AMBRONAY,None,None,None,00100139034,2024-01-23,11h27,Eau d'alimentation conforme aux limites de qua...,SYND. EAUX REGION D'AMBERIEU-EN-B,SERA - SYNDICAT DES EAUX DE LA REGION D'AMBERI...,SERA - SYNDICAT DES EAUX DE LA REGION D'AMBERI...,C,C,N,C,2024
1,001,001000003,01007,AMBRONAY,None,None,None,00100139393,2024-02-13,12h49,Eau d'alimentation conforme aux exigences de q...,SYND. EAUX REGION D'AMBERIEU-EN-B,SERA - SYNDICAT DES EAUX DE LA REGION D'AMBERI...,SERA - SYNDICAT DES EAUX DE LA REGION D'AMBERI...,C,C,C,C,2024
2,001,001000003,01007,AMBRONAY,001001304,TTP (CLG) AMBRONAY,100 %,00100139969,2024-03-13,10h46,Eau d'alimentation conforme aux exigences de q...,SYND. EAUX REGION D'AMBERIEU-EN-B,SERA - SYNDICAT DES EAUX DE LA REGION D'AMBERI...,SERA - SYNDICAT DES EAUX DE LA REGION D'AMBERI...,C,C,C,C,2024
3,001,001000003,01007,AMBRONAY,None,None,None,00100140209,2024-04-05,07h27,Eau d'alimentation conforme aux exigences de q...,SYND. EAUX REGION D'AMBERIEU-EN-B,SERA - SYNDICAT DES EAUX DE LA REGION D'AMBERI...,SERA - SYNDICAT DES EAUX DE LA REGION D'AMBERI...,C,C,C,C,2024
4,001,001000003,01007,AMBRONAY,None,None,None,00100140543,2024-05-13,12h43,Eau d'alimentation conforme aux exigences de q...,SYND. EAUX REGION D'AMBERIEU-EN-B,SERA - SYNDICAT DES EAUX DE LA REGION D'AMBERI...,SERA - SYNDICAT DES EAUX DE LA REGION D'AMBERI...,C,C,C,C,2024
5,001,001000003,01007,AMBRONAY,None,None,None,00100140886,2024-06-11,11h53,Eau d'alimentation conforme aux exigences de q...,SYND. EAUX REGION D'AMBERIEU-EN-B,SERA - SYNDICAT DES EAUX DE LA REGION D'AMBERI...,SERA - SYNDICAT DES EAUX DE LA REGION D'AMBERI...,C,C,C,C,2024
6,001,001000003,01007,AMBRONAY,001001304,TTP (CLG) AMBRONAY,100 %,00100141254,2024-07-08,09h50,Eau d'alimentation conforme aux exigences de q...,SYND. EAUX REGION D'AMBERIEU-EN-B,SERA - SYNDICAT DES EAUX DE LA REGION D'AMBERI...,SERA - SYNDICAT DES EAUX DE LA REGION D'AMBERI...,C,C,C,C,2024
7,001,001000003,01007,AMBRONAY,None,None,None,00100141669,2024-08-20,12h25,Eau d'alimentation conforme aux exigences de q...,SYND. EAUX REGION D'AMBERIEU-EN-B,SERA - SYNDICAT DES EAUX DE LA REGION D'AMBERI...,SERA - SYNDICAT DES EAUX DE LA REGION D'AMBERI...,C,C,C,C,2024
8,001,001000003,01007,AMBRONAY,None,None,None,00100142143,2024-09-11,10h09,Eau d'alimentation conforme aux exigences de q...,SYND. EAUX REGION D'AMBERIEU-EN-B,SERA - SYNDICAT DES EAUX DE LA REGION D'AMBERI...,SERA - SYNDICAT DES EAUX DE LA REGION D'AMBERI...,C,C,C,C,2024
9,001,001000003,01007,AMBRONAY,None,None,None,00100142530,2024-10-18,13h18,Eau d'alimentation non-conforme aux exigences ...,SYND. EAUX REGION D'AMBERIEU-EN-B,SERA - SYNDICAT DES EAUX DE LA REGION D'AMBERI...,SERA - SYNDICAT DES EAUX DE LA REGION D'AMBERI...,N,C,N,C,2024


2. Filtrer sur les prélévements non conformes en 2024



In [43]:
non_conforme_query = (
    (prelevements_table.plvconformitebacterio == "N") |
    (prelevements_table.plvconformitechimique == "N") |
    (prelevements_table.plvconformitereferencebact == "N") |
    (prelevements_table.plvconformitereferencechim == "N")
)

non_conformes_2024 = prelevements_2024.filter(
    non_conforme_query
)
f"En 2024, il y a eu {non_conformes_2024.count().execute():,.0f} prélévements non conformes au sens d'au moins une des variables suivantes : plvconformitebacterio, plvconformitechimique, plvconformitereferencebact, plvconformitereferencechim ([voir documentation](https://www.data.gouv.fr/fr/datasets/r/36afc708-42dc-4a89-b039-7fde6bcc83d8))".replace(",", " ") 


"En 2024  il y a eu 89 299 prélévements non conformes au sens d'au moins une des variables suivantes : plvconformitebacterio  plvconformitechimique  plvconformitereferencebact  plvconformitereferencechim ([voir documentation](https://www.data.gouv.fr/fr/datasets/r/36afc708-42dc-4a89-b039-7fde6bcc83d8))"

In [44]:
non_conformes_paris = non_conformes_2024.filter(
  (prelevements_table.nomcommuneprinc == "PARIS")
)
f"{non_conformes_paris.count().execute():,.0f} sur la ville de PARIS"

'32 sur la ville de PARIS'

## Selectionner des colonnes avant d'exécuter la requête

In [47]:
selected_columns = non_conformes_2024[["referenceprel", "dateprel", "nomcommuneprinc", "plvconformitebacterio"]]
selected_columns.execute()

,referenceprel,dateprel,nomcommuneprinc,plvconformitebacterio
0,00100142530,2024-10-18,AMBRONAY,N
1,00100141265,2024-07-10,MURS-ET-GELIGNIEUX,N
2,00100142451,2024-10-17,AMBUTRIX,N
3,00100143357,2024-11-22,ANGLEFORT,N
4,00100141194,2024-07-10,ARBOYS EN BUGEY,N
...,...,...,...,...
89294,97600032043,2024-08-12,BOUENI,C
89295,97600032044,2024-08-12,BOUENI,C
89296,97600031309,2024-01-11,PAMANDZI,C
89297,97600031337,2024-01-15,PAMANDZI,C


## Jointure
Joindre edc_prelevements et edc_resultats sur referenceprel pour obtenir les résultats associés à chaque prélèvement :

In [50]:

joined_data = non_conformes_2024.join(
    resultats_table,
    non_conformes_2024.referenceprel == resultats_table.referenceprel
)[["referenceprel", "dateprel", "nomcommuneprinc", "libmajparametre","insituana", "rqana", "cdunitereferencesiseeaux"]]

joined_data.execute()

,referenceprel,dateprel,nomcommuneprinc,libmajparametre,insituana,rqana,cdunitereferencesiseeaux
0,00800105683,2024-08-19,VILLERS-SUR-BAR,ASPECT (QUALITATIF),L,Aspect normal,SANS OBJET
1,00800105683,2024-08-19,VILLERS-SUR-BAR,BACT. ET SPORES SULFITO-RÉDU./100ML,L,0,n/(100mL)
2,00800105683,2024-08-19,VILLERS-SUR-BAR,CONDUCTIVITÉ À 25°C,L,580,µS/cm
3,00800105683,2024-08-19,VILLERS-SUR-BAR,CHLORE LIBRE,T,"<0,05",mg(Cl2)/L
4,00800105683,2024-08-19,VILLERS-SUR-BAR,CHLORE TOTAL,T,"0,07",mg(Cl2)/L
...,...,...,...,...,...,...,...
6024762,97400138800,2024-07-22,TAMPON (LE),ENTÉROCOQUES /100ML-MS,L,<1,n/(100mL)
6024763,97400138800,2024-07-22,TAMPON (LE),TITRE ALCALIMÉTRIQUE,L,"<2,0",°f
6024764,97400138800,2024-07-22,TAMPON (LE),TEMPÉRATURE DE L'EAU (DOM),T,"18,0",°C
6024765,97400138800,2024-07-22,TAMPON (LE),TEMPÉRATURE DE MESURE DU PH,T,"18,0",°C


## Groupby et aggregats
Nombre total de prélèvements non conforme par commune en 2024

In [38]:
agg_data = non_conformes_2024.group_by("nomcommuneprinc").aggregate(
    nb_prelevements_non_conformes=non_conformes_2024.referenceprel.count()
)
agg_data.execute().sort_values("nb_prelevements_non_conformes", ascending=False)

,nomcommuneprinc,nb_prelevements_non_conformes
1430,MARSEILLE,2016
12201,CAPESTERRE-BELLE-EAU,592
10366,AJACCIO,374
12211,SAINT-DENIS,344
4116,PREMESQUES,294
...,...,...
1,BEAUMONT-DU-VENTOUX,1
13769,PIETRA-DI-VERDE,1
13766,PIAZZALI,1
14,PORT-DE-PILES,1
